# Neutron Data

This is the continuation from [Multi-dimensional datasets](https://scipp.readthedocs.io/en/latest/tutorials/multi-d-datasets.html) tutorial.
Note that this notebooks requires [Mantid](https://www.mantidproject.org/Main_Page) and data files that are, e.g., contained in the [Docker](https://hub.docker.com/r/scipp/scipp-jupyter-demo) image of scipp.
Therefore, outputs are unfortunately not available on readthedocs.

In [ ]:
import numpy as np
import scipp as sc
from scipp import Dim
from scipp.plot import plot

## Loading Nexus files

Scipp does not support native loading of [Nexus](https://www.nexusformat.org/) files at this point.
However, it can leverage Mantid to do this:

In [ ]:
events = sc.Dataset()
# Load only a single bank to reduce memory consumption, so this runs on a laptop
events["sample"] = sc.neutron.load(
    filename='PG3_4844_event.nxs',
    load_pulse_times=True,
    mantid_args={'BankName': 'bank184',
                 'LoadMonitors': True}) # Mantid does not load monitors by default
events["vanadium"] = sc.neutron.load(
    filename='PG3_4866_event.nxs',
    load_pulse_times=False,
    mantid_args={'BankName': 'bank184',
                 'LoadMonitors': True})

Note that internally this calls `Load` or `LoadEventNexus` provided by Mantid.
Scipp then converts from Mantid's `EventWorkspace` and `Workspace2D` to `DataArray`.
Currently not all information from the Mantid workspaces is preserved in the data array.

## Understanding the contents of the created dataset

The dataset with loaded sample and vanadium data looks as follows:

In [ ]:
events

We give a short discussion of each of the entries to familiarize ourselves with how data from a Mantid workspace is mapped onto a data array or dataset.

### Dimensions and coordinates

#### Spectrum

In most Mantid workspaces each spectrum corresponds to data measured at a detector pixel, i.e., at a specific position or region in space.
If that is the case, scipp used `Dim.Spectrum` for this dimension.

Note that using the generic `Dim.Spectrum` should be avoided in other cases.
For example, after converting data to `Q` we need to avoid having "compatible" dimensions of a data and would use `Dim.Q`.
The double meaning of what a "spectrum" actually is in Mantid is thus avoided.

The spectrum dimension comes with a coordinate:

In [ ]:
events.coords[Dim.Spectrum]

#### Time-of-flight

In contrast to a `EventWorkspace` in Mantid, a dataset does not necessarily come with a time-of-flight (TOF) coordinate (bin edges) on top of the TOF values for the events.
Therefore `Dim.Tof` does not have a corresponding *dense* coordinate.
See below for *sparse* TOF coordinates.

### Labels

Scipp stores auxiliary "coordinate" information as labels.
Labels or coords (and not attributes) are used to ensure that information is compatible in operations involving multiple data arrays or dataset.

This actually happended internally when we first loaded the files for sample and vanadium and inserted them into the same dataset:
If the files had had different spectrum numbers or spectrum positions the insertion of the `'vanadium'` data would have failed due to incompatible coordinates or labels.

#### Position

Positions are an auxiliary coordinate for `Dim.Spectrum`, in other words they could be used to "label" the spectrum coordinate, e.g., in an a plot.
The main purpose of storing postions as labels instead of, e.g., attributes is to ensure that operations between data with mismatching detector positions fail and thus prevent mistakes.

In [ ]:
events.labels['position']

The position coordinate stores the positions of all spectra.
Each position is a 3-component vector (X, Y, Z).

#### Beamline geometry

Apart from positions of spectra we require additional geometry information for various components in a neutron beamline (instrument).
This is stored as labels such as `'source_position'` and `'sample_position'`.
We instead recommend the use of helper functions such as `sample_position` which will be discussed below:

In [ ]:
events.labels['sample_position'].value

We emphazise the importance of storing this information as labels.
This ensures that we cannot accidentally combine data obtained with, e.g., different sample positions.

*Bonus note:
If we ever* **do** *want to combine data with different samples we can either remove this information from the dataset, or change it to an* **attribute**.

For convenient and standardized access, as well as access of derived information such as scattering angles, a number of helper functions is provided.
For example:

In [ ]:
sc.neutron.sample_position(events).copy() # copy is a temporary hack since const view lacks some properties

In [ ]:
sc.neutron.scattering_angle(events)

For a full list of available beamline-geometry helpers please refer to the [documentation](https://scipp.readthedocs.io/en/latest/additional-modules/scipp-neutron.html#beamline-geometry).

*Bonus note:
 For the most part, the structure of `ComponentInfo` (and `DetectorInfo`) in Mantid is easily represented by a `Dataset`, i.e., very little change is required.
 For example, scanning is simply handled by an extra dimension of, e.g., the position and rotation variables.
 By using `Dataset` to handle this, we can use exactly the same tools and do not need to implement or learn a new API.*
 
### Attributes

Attributes allow for storing information with data in a way that leaves it untouched in normal operations.
This is used for storing meta-data that does not require processing alongside normal data.
For neutron data key examples are: run information, sample information, and neutron monitors.
Scipp simply stores Mantid's `Run` and `Sample` objects in the `"run"` and `"sample"` attributes.
Monitors are stored in the `"monitors"` attribute.

In [ ]:
sc.to_html(events['sample'].attrs['run'])
sc.to_html(events['sample'].attrs['sample'])
sc.to_html(events['sample'].attrs['monitors'])

Note that these are attributes of the sample (or vanadium).
A dataset itself can also have attributes, but in this case it does not.
Each of the attributes above is a 0-D variable, the easiest way to access this value is the `value` property:

In [ ]:
mantid_run = events['sample'].attrs['run'].value
print('The run contains the following properties:\n{}\n'.format(mantid_run.keys()))

mantid_sample = events['sample'].attrs['sample'].value
print('Sample name: {}\n'.format(mantid_sample.getName()))

# Each monitor (just 1 in this case) is stored as a data array
print('Monitor:')
sc.to_html(events['sample'].attrs['monitors'].value)

### Event data

Neutron events are stored as **sparse data** in contrast to the regular gridded ("dense") data of, e.g., histogrammed data.
See [the scipp documentation](https://scipp.readthedocs.io/en/latest/user-guide/sparse-data.html) for more information.

The number of neutrons detected at each position is different and thus scipp has no fixed definition for the length of the "sparse" dimension.
This looks as follows:

In [ ]:
sc.show(events[Dim.Spectrum, 10:20])

This data structure is to be interpreted as follows:

- Each position sees a different number of events, and events arrive at random time.
  Therefore, there is a time-of-flight for *every pixel*, for *every event*, and for *every data item* (`'vanadium'` and `'sample'`).
  This **sparse coordinate** has the following properties:
  - The sparse coord for `Dim.Tof` is associated with a data item and is not global for the dataset.
  - The sparse coord for `Dim.Tof` depends on `Dim.Spectrum`.
  - The sparse coord for `Dim.Tof` has a different length for each spectrum.
- Extra information such as pulse times are stored as sparse labels.
  What was said above for sparse coords also applies to sparse labels.
  The length at each position matches the corresponding length of the sparse coordinate.
- Values and variances are optional.
  They would represent weight and weight uncertainties of events.
  If they are not present an implicit weight of `1` is assumed, i.e., each coord value corresponds to a single neutron.

The time-of-flight values for an individual pixel could be accessed as follows:

In [ ]:
events['sample'].coords[sc.Dim.Tof][sc.Dim.Spectrum, 10].values

### From events to histogram

We histogram the event data:

In [ ]:
bins = sc.Variable([Dim.Tof], values=np.arange(1000.0, 20000.0, 50.0), unit=sc.units.us)
d = sc.histogram(events, bins)
d

In [ ]:
plot(d['sample'])

### Fake instrument view

Just for fun, we can quickly generate a crude "instrument view".
In this case this works since we have only a single panel.
If there were multiple panels, they could be handled as an extra dimension.

In [ ]:
panel = sc.Dataset()
# 154 and 7 are the extents of the panel
panel['sample'] = sc.reshape(d['sample'].data, [Dim.X, Dim.Y, Dim.Tof], (154,7,379))
panel.coords[Dim.Tof] = d.coords[Dim.Tof]
# Note that the scale is meaningless, could use real instrument parameters
panel.coords[Dim.X] = sc.Variable([Dim.X], values=np.arange(154))
panel.coords[Dim.Y] = sc.Variable([Dim.Y], values=np.arange(7))
# Move TOF slider around 12000 to see diffraction lines moving across the panel
plot(panel[Dim.Tof, 180:260], axes=[Dim.Tof, Dim.Y, Dim.X])

### Monitors

If loaded, monitors are available as an attributes named `"monitors"`.
For demonstration purposes we can add some fake monitors to demonstrate the versatility of `Dataset`.
Storing each monitor as a separate variable that contains a nested data array gives us complete freedom an flexibility.
Another approach would be to use a single 1-D variable containing monitor data arrays as elements.

In [ ]:
# Histogram-mode beam monitor
edges = np.arange(0.0, 20000.0, 1000.0)
counts = np.random.rand(len(edges)-1)
beam = sc.DataArray(
    data=sc.Variable([Dim.Tof], values=counts, variances=counts, unit=sc.units.counts),
    coords={Dim.Tof: sc.Variable([Dim.Tof], values=edges)})

# Event-mode transmission monitor
events = sc.Variable([Dim.Tof], shape=[sc.Dimensions.Sparse])
events.values = np.random.rand(123456)
transmission = sc.DataArray(coords={Dim.Tof: events})

# Beam profile monitor
profile = sc.DataArray(
    data=sc.Variable([Dim.Y, Dim.X], values=np.random.rand(20, 20), unit=sc.units.counts),
    coords={
        Dim.X: sc.Variable([Dim.X], values=np.arange(-0.1, 0.11, 0.01)),
        Dim.Y: sc.Variable([Dim.Y], values=np.arange(-0.1, 0.11, 0.01))
    })
for i in 1,2,3,4:
    profile[Dim.X, i:-i][Dim.Y, i:-i] += 1.0 * sc.units.counts

plot(profile)

In [ ]:
sc.Variable(value=transmission)
d['sample'].attrs['transmission'] = sc.Variable(value=transmission)
d['sample'].attrs['beam'] = sc.Variable(value=beam)
d['sample'].attrs['profile'] = sc.Variable(value=profile)
d

### Exercise 1
Normalize the sample data to the first (and only) monitor in the `"monitors"` attribute.
This involves a couple of steps:
1. Normalization in time-of-flight does not make sense. Convert to, e.g., wavelength.
2. The data and the monitor have mismatching bins. Rebin to common binning.
3. Normalize. 

### Solution 1

In [ ]:
sample = sc.neutron.convert(d['sample'], Dim.Tof, Dim.Wavelength)
# Using bins of first spectrum here, other choices are just as valid
sample = sc.rebin(sample, Dim.Wavelength, sample.coords[Dim.Wavelength][Dim.Spectrum, 0])

mon = d['sample'].attrs['monitors'].value
mon = sc.neutron.convert(mon, Dim.Tof, Dim.Wavelength)
mon = sc.rebin(mon, Dim.Wavelength, sample.coords[Dim.Wavelength])

plot(mon)                             
sample_over_beam = sample / mon
plot(sample_over_beam)

We could now continue with the normalized data, but in the following we do not.

### Adding new dimensions

In [ ]:
sample = d['sample']
temp_scan = sc.concatenate(sample, sample * (0.8 * sc.units.dimensionless), Dim.Temperature)
temp_scan = sc.concatenate(temp_scan, temp_scan * (0.64 * sc.units.dimensionless), Dim.Temperature)
temp_scan.coords[Dim.Temperature] = sc.Variable([Dim.Temperature], values=[4.3, 100.0, 180.0, 273.0])

In [ ]:
plot(temp_scan[Dim.Spectrum, 20:], axes=[Dim.Tof, Dim.Temperature, Dim.Spectrum])

In [ ]:
plot(temp_scan[Dim.Spectrum, 500], collapse=Dim.Tof)

### Unit conversion

Unit conversion is available in the `scipp.neutron` submodule.
Converting a data array or dataset to a different unit implies changing one of the dimensions and its coordinate.
Conversion can also be done with sparse data (events), but we are using the histogrammed data here:

In [ ]:
d = sc.neutron.convert(d, Dim.Tof, Dim.DSpacing)

In [ ]:
# Plotting cannot handle ragged coordinates at this point, rebin to edges of first spectrum
d = sc.rebin(d, Dim.DSpacing, d.coords[Dim.DSpacing][Dim.Spectrum, 0])

In [ ]:
plot(d)

### Summing and normalizing

In [ ]:
summed = sc.sum(d, Dim.Spectrum)
plot(summed)

In [ ]:
normalized = summed['sample'] / summed['vanadium']
plot(normalized)

### Exercise 2 (advanced)

Instead of loading only a single bank, load multiple, e.g., `bank124`, `bank144`, `bank164`, and `bank184`.
Modify everything in this notebook to work with the new multi-bank data, obtaining a separate focussed diffraction spectrum for each bank.

There is more than one option to solve this:
1. Concatenate the loaded data into a single dataset, resulting in more or larger dimensions.
2. Merge the loaded data into a single dataset, resulting in differently named variables for each bank.
3. Call the existing code as-is for each bank, working, e.g., for a Python `list` of datasets.

Each of the approaches has its advantages and drawbacks.

Here we recommend option 1, which in itself can be implemented in one of two ways:
- Concatenate along a new dimension (`Dim.Bank` is not supported currently, use, e.g., `Dim.Row` instead).
- Concatenate along the existing dimension `Dim.Spectrum`.

*Note: You will likely experience some small problems with plotting, in particular issues with multi-dimensional coordinates in the first case (we suggest to slice manually until this is supported), and large gaps in the second case (can be avoided by adding a helper-coordinate).*

*Bonus note for option 3: Unlike Mantid workspaces, datasets can safely be used in combination with Python containers. Do not try this with workspaces, since they are entangled with the `AnalysisDataService`.*
